In [13]:
import pandas as pd #type: ignore
df = pd.read_csv("adib-feedbacks-data.csv")
df.head()

,annotation_id,annotator,brand_labels,category_v2,category_v3,city,country,created_at,description,facebook_url,...,subscription_service,task_status,twitter_url,twitter_url_dropdown,twitter_url_from_google,updated_at,url,website_from_google,website_from_perplexity,website_url_dropdown
0,142573,19,Dubai Taxi Corporation,Transportation,Transportation,Dubai,United Arab Emirates,2025-04-28T07:19:59.880514Z,Dubai Taxi Corporation is a government entity ...,NaN,...,NaN,Accept,NaN,https://twitter.com/DTCUAE,https://twitter.com/DTCUAE,2025-04-28T07:19:59.880524Z,https://www.dubaitaxi.ae/en,https://www.dubaitaxi.ae/en,NaN,https://www.dubaitaxi.ae/en
1,142583,19,Meta,Entertainment,Entertainment,Dubai,United Arab Emirates,2025-04-28T10:38:03.042976Z,ROI-driven performance marketing agency specia...,NaN,...,NaN,NaN,NaN,NaN,https://sproutsocial.com/insights/best-times-t...,2025-04-28T10:38:03.042986Z,https://metasocial.ae,https://metasocial.ae/,NaN,NaN
2,142584,19,Yas Mart Shopping Center,Shopping,Shopping,Abu Dhabi,United Arab Emirates,2025-04-28T10:38:25.734126Z,Yas Mart is a hypermarket offering a range of ...,NaN,...,NaN,NaN,NaN,NaN,https://us.twsteel.com/,2025-04-28T10:38:25.734134Z,NaN,https://www.instagram.com/yasmart.ae/?hl=en,NaN,NaN
3,142586,19,Ounass,Shopping,Shopping,Dubai,United Arab Emirates,2025-04-28T10:38:38.832365Z,Ounass is the Middle East’s premier luxury onl...,NaN,...,NaN,NaN,NaN,NaN,https://twitter.com/aaital_khosla,2025-04-28T10:38:38.832373Z,https://www.ounass.com,https://www.ounass.com/,NaN,NaN
4,142577,19,Fruits Monesh Juices,Dining,Food & Beverage,Dubai,United Arab Emirates,2025-04-28T10:37:29.459516Z,A juice shop offering various beverages.,NaN,...,NaN,NaN,NaN,NaN,https://www.monashfodmap.com/ibs-central/i-hav...,2025-04-28T10:37:29.459525Z,NaN,https://ae.linkedin.com/in/aliahmed621,NaN,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   annotation_id              760 non-null    int64  
 1   annotator                  760 non-null    int64  
 2   brand_labels               760 non-null    object 
 3   category_v2                751 non-null    object 
 4   category_v3                750 non-null    object 
 5   city                       742 non-null    object 
 6   country                    749 non-null    object 
 7   created_at                 760 non-null    object 
 8   description                746 non-null    object 
 9   facebook_url               87 non-null     object 
 10  facebook_url_dropdown      1 non-null      object 
 11  facebook_url_from_google   732 non-null    object 
 12  id                         760 non-null    int64  
 13  instagram_url              233 non-null    object 

In [15]:
new_df = df[["name", "url"]]
new_df

,name,url
0,Dubai Taxi Corporation,https://www.dubaitaxi.ae/en
1,Meta Social,https://metasocial.ae
2,Yas Mart,NaN
3,Ounass,https://www.ounass.com
4,Fruits Monesh,NaN
...,...,...
755,Loyal Mini Mart,NaN
756,Grill It,NaN
757,Giorgio Armani,https://locations.armani.com/en/emporio-armani...
758,Sunglass Hut,https://ae.sunglasshut.com


In [16]:
# number of brands is 27,838 Brands

numberOfBrands = new_df["name"].count()
numberOfBrands

np.int64(760)

In [17]:
# number of brands with a website link is 20,868 Brands

numberOfBrandsWithAWebsiteLink = new_df["url"].count()
numberOfBrandsWithAWebsiteLink

np.int64(538)

In [18]:
# Brands with no website link is 6,970 Brands

brandsWithNoWebsiteLink = new_df["name"].count() - new_df["url"].count()
brandsWithNoWebsiteLink

np.int64(222)

In [ ]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

TIMEOUT = 5
MAX_WORKERS = 100

def is_link_working(url):
    if not isinstance(url, str) or not url.startswith("http"):
        return False
    try:
        headers = {
            'User-Agent': (
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                'AppleWebKit/537.36 (KHTML, like Gecko) '
                'Chrome/120.0.0.0 Safari/537.36'
            )
        }
        resp = requests.get(url, timeout=TIMEOUT, allow_redirects=True, headers=headers)
        return 200 <= resp.status_code < 400
    except Exception:
        return False

def check_links_parallel(df, url_column='url', max_workers=MAX_WORKERS):
    results = [None] * len(df)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_idx = {
            executor.submit(is_link_working, df.iloc[i][url_column]): i
            for i in range(len(df))
        }

        for future in tqdm(as_completed(future_to_idx), total=len(future_to_idx), desc="🔗 Checking links"):
            idx = future_to_idx[future]
            try:
                results[idx] = future.result()
            except Exception:
                results[idx] = False

    df['link_working'] = results
    return df

if __name__ == "__main__":
    new_df = pd.read_csv("adib-feedbacks-data.csv")
    new_df = new_df.dropna(subset=['name', 'url'])
    checked_df = check_links_parallel(new_df)
    working = checked_df['link_working'].sum()
    total = len(checked_df)
    print(f"\n✅ Working: {working}")
    print(f"❌ Not working: {total - working}")
    checked_df[['name', 'url', 'link_working']].to_excel("URL_CHECKER_RESULTS/adib-crawl_URL_CHECKER_RESULTS.xlsx", index=False)

🔗 Checking links: 100%|██████████| 538/538 [01:01<00:00,  8.81it/s] 


✅ Working: 466
❌ Not working: 72


In [20]:
# Performance 

Accuracy = (working / total) * 100

print(f"Accuracy: {Accuracy:.2f}%")

Accuracy: 86.62%


before: 67.10%

after: 87.17%